Stability AI (авторы StableDiffusion) анонсировали аналог Github Copilot — [StableCode](https://stability.ai/blog/stablecode-llm-generative-ai-coding) .

Нейросеть доступна в трёх вариантах: базовый, модель для инструкций и расширенная с контекстным окном до 16 тысячи токенов. Из языков программирования нейросеть поддерживает такие как Python, Java, JavaScript, Go, C, C++.

Модели же доступны на Huggin Face:

— [stablecode-completion-alpha-3b-4k](https://huggingface.co/stabilityai/stablecode-completion-alpha-3b-4k)

— [stablecode-completion-alpha-3b](https://huggingface.co/stabilityai/stablecode-completion-alpha-3b)

— [stablecode-instruct-alpha-3b](https://huggingface.co/stabilityai/stablecode-instruct-alpha-3b)

Рассмотрим возможность использования данной модели

Установим `transformers`



In [2]:
#!pip install transformers==4.29.2
!pip install transformers==4.31.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.2 MB/s eta 0:00:00


Установим инструмент для работы с Haggingface

In [3]:
!pip install huggingface_hub

Для получения доступа надо сформировать токен на [Haggingface](https://huggingface.co/settings/tokens)


Сохраним токен

In [4]:
!python -c "from huggingface_hub.hf_api import HfFolder; HfFolder.save_token('<Ваш токен>')"

Зарегистрируемся в Haggingface с вводом токена

In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("stabilityai/stablecode-instruct-alpha-3b", use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(
  "stabilityai/stablecode-instruct-alpha-3b",
  trust_remote_code=True,
  torch_dtype="auto",
  use_auth_token=True,
)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [20]:
# @title Load question  { display-mode: "form" }
model.cuda()
question = "Generate a python function to find number of CPU cores"# @param {type:"string"}
inputs = tokenizer("###Instruction\n"+question+"###Response\n",
                   return_tensors="pt",
                   return_token_type_ids=False).to("cuda")

tokens = model.generate(
  **inputs,
  max_new_tokens=500,
  temperature=0.2,
  do_sample=True,
)

answer = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(answer)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


###Instruction
Generate a python function to find number of CPU cores###Response
def get_num_cores():
    """
    This function will return the number of CPU cores in the system
    """
    # Use the Linux command "lscpu" to get the output
    output = os.popen("lscpu").read()
    # Split the output into lines
    lines = output.split("\n")
    # Get the number of CPU cores
    num_cores = len(lines) - 1
    # Return the number of CPU cores
    return num_cores


Проверим сгенеренный код

In [9]:
def get_cpu_count():
    """
    This function will return the number of CPU cores
    installed in the system
    """
    import multiprocessing
    return multiprocessing.cpu_count()

In [10]:
get_cpu_count()

2

Скачивать постоянно модель не совсем удобно. По этому рассмотрим сохранение модели и работу с сихраненной моделью

создадим папку для хранения модели

In [11]:
!mkdir model_files

NotImplementedError: ignored

In [12]:
# Save huggingface model files to working directorty
model.save_pretrained('model_files')
tokenizer.save_pretrained('model_files')


('model_files/tokenizer_config.json',
 'model_files/special_tokens_map.json',
 'model_files/tokenizer.json')

In [13]:
tokenizer1 = AutoTokenizer.from_pretrained("stabilityai/stablecode-instruct-alpha-3b",
                                           use_auth_token=True,
                                           cache_dir = 'model_files')
model1 = AutoModelForCausalLM.from_pretrained(
  "stabilityai/stablecode-instruct-alpha-3b",
  trust_remote_code=True,
  torch_dtype="auto",
  use_auth_token=True,
  cache_dir = 'model_files',
)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [14]:
model1.cuda()
inputs = tokenizer1("###Instruction\nGenerate a python function to find number of CPU cores###Response\n",
                   return_tensors="pt",
                   return_token_type_ids=False).to("cuda")
tokens = model1.generate(
  **inputs,
  max_new_tokens=500,
  temperature=0.2,
  do_sample=True,
)
print(tokenizer.decode(tokens[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


###Instruction
Generate a python function to find number of CPU cores###Response
def get_num_cores():
    """
    This function returns the number of CPU cores in the system
    """
    # Use the cpuinfo library to get the number of cores
    from cpuinfo import cpuinfo
    info = cpuinfo.get_cpu_info()
    num_cores = info['count']
    return num_cores


И проверим код сгенеренный

In [17]:
def get_num_cores():
    """
    This function returns the number of CPU cores in the system
    """
    # Use the cpuinfo library to get the number of cores
    from cpuinfo import cpuinfo
    info = cpuinfo.get_cpu_info()
    num_cores = info['count']
    return num_cores

get_num_cores()

2